In [4]:
!fusermount -u drive
from google.colab import drive
drive.mount('/content/drive')

import os

# Replace 'your_dataset.zip' with the actual filename
train_tar_path = '/content/drive/MyDrive/ImageNet Dataset/train.tar'
val_tar_path = '/content/drive/MyDrive/ImageNet Dataset/val.tar'

# Directory to extract the datasets
extract_dir = '/content/dataset/'

# Create directories if they don't exist
os.makedirs(extract_dir, exist_ok=True)

import tarfile

# Function to extract tar files
def extract_tar(file_path, dest_path):
    with tarfile.open(file_path, 'r') as tar_ref:
        tar_ref.extractall(dest_path)

# Extract training and validation datasets
extract_tar(train_tar_path, extract_dir)
extract_tar(val_tar_path, extract_dir)

# List the contents of the extracted directories
print("Extracted files:")
print(os.listdir(extract_dir))

Mounted at /content/drive
Extracted files:
['val', 'train']


In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths to the extracted datasets
train_dir = os.path.join(extract_dir, 'train')
val_dir = os.path.join(extract_dir, 'val')

# Image data generator with normalization
datagen = ImageDataGenerator(rescale=1.0/255.0)

# Training data generator
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Validation data generator
validation_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 1 classes.


In [6]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Load pre-trained MobileNet without the top layer
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Set training parameters
epochs = 10
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = validation_generator.samples // validation_generator.batch_size

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_steps
)


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 463/3125 ━━━━━━━━━━━━━━━━━━━━ 59:55 1s/step - accuracy: 0.1176 - loss: 4.3842

In [ ]:
# Evaluate on validation data
loss, accuracy = model.evaluate(validation_generator, steps=validation_steps)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')
